https://opencritic.com/browse/all/all-time/num-reviews?page=3

https://rapidapi.com/console/10520148/billing/subscriptions-and-usage

In [140]:
from requests import get
import time
import pandas as pd

In [83]:

allgames_url = "https://opencritic-api.p.rapidapi.com/game"
headers = {"x-rapidapi-host": "opencritic-api.p.rapidapi.com",
          "x-rapidapi-key": "33ada1ad65msh5cd9bcf43c10386p119501jsneb46151fedcf"
          }
queries={"platforms":"all", "sort":"score","skip":0}
resp = get(url=allgames_url, headers=headers, params=queries)
#resp = get(url=url)

In [84]:
print(dir(resp))

['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']


In [85]:
items = resp.json()
print("Example Item:")
print(items[0])
#display(items)


#items.json()

Example Item:
{'images': {'box': {'og': 'game/4504/o/1oc8VdkN.jpg', 'sm': 'game/4504/DTh6jjTo.jpg'}, 'banner': {'og': 'game/4504/o/oh38qUAE.jpg', 'sm': 'game/4504/BZrxOMDi.jpg'}}, 'percentRecommended': 98.01324503311258, 'numReviews': 157, 'topCriticScore': 96.8108108108108, 'tier': 'Mighty', 'name': 'Super Mario Odyssey', 'Platforms': [{'id': 32, 'name': 'Nintendo Switch', 'shortName': 'Switch', 'releaseDate': '2017-10-27T00:00:00.000Z'}], 'Genres': [{'_id': '5d8678fd93765d677877b650', 'id': 76, 'name': 'Adventure'}, {'_id': '5d8678fd93765d677877b64f', 'id': 82, 'name': 'Platformer'}], 'id': 4504, 'firstReleaseDate': '2017-10-27T00:00:00.000Z', 'tags': [{'_id': '6699556e534cfd6134a07934', 'displayOnFilter': True, 'isActive': True, 'name': 'Platformer', 'initialPopularity': 947, '__v': 0, 'tagId': 'platformer', 'id': '6699556e534cfd6134a07934'}, {'_id': '6699556b534cfd6134a078e0', 'displayOnFilter': True, 'isActive': True, 'name': 'Adventure', 'initialPopularity': 3280, '__v': 0, 'tagI

In [141]:
def parseValues(response_text):
    values = {}
    for i in response_text:
        #i_values = {}
        #Extract the game id and name
        values[i['id']] = {'name':i['name'],"n_reviews":i['numReviews']}
    return(values)

def saveResults(results, save_path, verbose=True):
    if verbose:
        print(f"Saving results to {save_path}")
    df = pd.DataFrame.from_dict(results, orient="index")
    df.to_parquet(save_path,index=True)

class OpenCriticAPI:
    def __init__(self, endpoint, query_params=None, limit_pages=1):
        self.url = f"https://opencritic-api.p.rapidapi.com/{endpoint}"
        self.headers = {"x-rapidapi-host": "opencritic-api.p.rapidapi.com",
                "x-rapidapi-key": "33ada1ad65msh5cd9bcf43c10386p119501jsneb46151fedcf"
            }
               
        if query_params:
            self.params = query_params
        else:
            self.params = queries={"platforms":"all", "sort":"num-reviews","skip":0}
      
    def callOne(self,pagination=None):
        if pagination:
            new_params = self.params
            new_params['skip'] = (pagination - 1) * 20
            self.response = get(url=self.url, headers=self.headers, params=new_params)
        else:
            self.response = get(url=self.url, headers=self.headers, params=self.params)
        return(self)
    
    def callAll(self, results_path, max_pages=1, rate_limit=3, save_on_error=True):
        results = {}
        pages_read = 0
        
        for page_number in range(max_pages):
            #Set our page number
            page_number = pages_read + 1
            
            try:
                #call callOne
                response = self.callOne(pagination=page_number)
                #increment pages_read
                pages_read += 1
                print(f"Pages read: {pages_read}")
                status_code = response.response.status_code
                print(f"Status Code: {status_code}")
                
                if status_code != 200:
                    print(f"Non-200 Response: {response.response.text}")
                    if save_on_error and results:
                        saveResults(results=results, save_path=results_path)
                    break
                    
                new_results = parseValues(response.response.json())
                #merge dictionaries
                results = {**results, **new_results}

                #sleep to avoid rate limit
                time.sleep(1/rate_limit)
            
            except Exception as e:
                print(f"Exception occurred: {e}")
                if save_on_error and results:
                    saveResults(results=results, save_path=results_path)
                break
                        
        return(results)
    
    def parse(self):
        values = parseValues(self.response.json())
        return(values)
    
    

In [142]:
ocapi = OpenCriticAPI(endpoint="game")
test_res = ocapi.callAll(results_path='./data/ocgames.parquet', max_pages=20)
print(test_res)

Pages read: 1
Status Code: 429
Non-200 Response: {"message":"You have exceeded the DAILY quota for Requests on your current plan, BASIC. Upgrade your plan at https:\/\/rapidapi.com\/opencritic-opencritic-default\/api\/opencritic-api"}
{}
